This contains selenium codes to scrap company urls from [trustpilot.com](https://fr.trustpilot.com/)

## Why Selenium :-
    

# automation for "Facebook Login Page"

    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys

    user_name = 'your_username'
    password = 'your_password'

    driver = webdriver.Chrome('/usr/bin/chromedriver')
    driver.get('https://www.facebook.com')

    element = driver.find_element_by_id('email')
    element.send_keys(user_name)

    element = driver.find_element_by_id('pass')
    element.send_keys(password)
    element.send_keys(Keys.RETURN)


In [13]:
%config Completer.use_jedi=False

In [21]:
import json
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from tqdm.notebook import tqdm

In [32]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome('/usr/bin/chromedriver', options=options)

timeout = 3

In [22]:
base_url = "https://trustpilot.com"

In [ ]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')


In [ ]:
data = {}

soup = get_soup(base_url + '/categories')

for categories in soup.find_all('div' ,{'class':'subCategory___BRUDy'}):
    name = categories.find('a',{'class': 'navigation___2Efid'}).text
    name = name.strip()
    data[name] = {}
    
    sub_categories = categories.find('div', {'class': 'subCategoryList___r67Qj'})

    for sub_category in sub_categories.findAll('div', {'class': 'subCategoryItem___3ksKz'}):
        sub_category_name = sub_category.find('a', {'class': 'navigation___2Efid'}).text 
        sub_category_uri = sub_category.find('a', {'class': 'navigation___2Efid'})['href'] 
        data[name][sub_category_name] = sub_category_uri


data

len(data)

In [61]:
def extract_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('//a[@class="wrapper___2rOTx"]')
    urls = [a.get_attribute('href') for a in a_list]
    dedup_urls = list(set(urls))
    return dedup_urls

    dedup_urls[1]

    driver.get(url)
    a_list = driver.find_elements_by_xpath('//a[@class="wrapper___2rOTx"]')
    urls = [a.get_attribute('href') for a in a_list]
    dedup_urls = list(set(urls))
    print(dedup_urls)
    print(len(dedup_urls))

    url = base_url + data['Electronics & Technology']['Computers & Phones'] + "?numberofreviews=0&timeperiod=0&status=all"
    url

In [63]:
def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        return False, None

    button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
    print(button)

    next_url = base_url + data['Electronics & Technology']['Computers & Phones'] + "?numberofreviews=0&timeperiod=0&status=all" + f'&page={3}'
    print(next_url)

In [64]:
company_urls = {}
for category in tqdm(data):
    for sub_category in tqdm(data[category], leave=False):
        company_urls[sub_category] = []

        url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all"
        driver.get(url)
        try: 
            element_present = EC.presence_of_element_located(
                (By.CLASS_NAME, 'wrapper___2rOTx'))
            
            WebDriverWait(driver, timeout).until(element_present)
        except:
            pass
    
        next_page = True
        c = 1
        while next_page:
            extracted_company_urls = extract_company_urls_form_page()
            company_urls[sub_category] += extracted_company_urls
            next_page, button = go_next_page()
            
            if next_page:
                c += 1
                next_url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all" + f'&page={c}'
                driver.get(next_url)
                try: 
                    element_present = EC.presence_of_element_located(
                        (By.CLASS_NAME, 'wrapper___2rOTx'))
                    
                    WebDriverWait(driver, timeout).until(element_present)
                except:
                    pass


In [65]:
with open('company_urls_en', 'w') as f:
    json.dump(company_urls, f)

In [70]:
consolidated_data = []

for category in data:
    for sub_category in data[category]:
        for url in company_urls[sub_category]:
            consolidated_data.append((category, sub_category, url))

df_consolidated_data = pd.DataFrame(consolidated_data, columns=['category', 'sub_category', 'company_url'])

df_consolidated_data.to_csv('company_urls_en.csv', index=False)

In [73]:
df_consolidated_data.tail()

,category,sub_category,company_url
12994,Vehicles & Transportation,Vehical Rental,https://www.trustpilot.com/review/www.dcscoote...
12995,Vehicles & Transportation,Vehical Rental,https://www.trustpilot.com/review/eurocarconne...
12996,Vehicles & Transportation,Vehical Rental,https://www.trustpilot.com/review/www.cruiseam...
12997,Vehicles & Transportation,Vehical Rental,https://twitter.com/Trustpilot
12998,Vehicles & Transportation,Vehical Rental,https://www.trustpilot.com/about
